# Store file for animated gif

Then use Ovito. Here is a tutorial https://www.ovito.org/docs/current/tutorials/turntable_animation.html#tutorials-turntable-animation

NB: store png and separately create gif for better quality

In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
import numpy as np
from rigid_flows.systems.watermodel import WaterModel

temp = 100
water_type = 'tip4pew'
rigidWater = True
ice_type = 'XI'
n_waters = 128

# path = ''
path = '/group/ag_cmb/scratch/minvernizzi/so3-flow/ice_MDdata/'
info = f'ice{ice_type}_T{temp}_N{n_waters}'
if water_type != 'tip4pew':
    info = f'{water_type}_{info}'
if not rigidWater:
    info = f'flex_{info}'
model = WaterModel.load_from_json(f'{path}model-{info}.json')
# data = np.load(f'{path}MDtraj-{info}.npz')
# pos = data['pos']
# box = data['box']
# ene = data['ene']

In [19]:
MDpos = np.load(f'{path}MDtraj-{info}.npz')['pos'][::1000]
MDpos += np.diag(model.box)/16 #shift crystal more to center

In [24]:
#save file for ovito
mask_vs = np.tile(np.concatenate((3*[True], (model.n_sites-3)*[False])), model.n_waters)
traj = model.get_mdtraj(MDpos[:,mask_vs], None, model.generate_mdtraj_topology(model.n_waters, 3))

filename = 'gif-N128_T100.pdb'
traj.save(filename)

### mdtraj only stores box info for the first frame
### if no box info is available, ovito guesses the box size from positions
### so we must add the box info to each frame
with open(filename, 'r') as f:
    lines = f.readlines()
assert lines[1][:5] == 'CRYST'
print(lines[1], end='')

count = 0
for i in range(len(lines)):
    if lines[i][:5] == 'MODEL':
        if lines[i-1][:5] != 'CRYST':
            lines[i] = lines[1] + lines[i]
            count += 1
print(f'- box info added {count} times')

with open(filename, "w") as f:
    lines = "".join(lines)
    f.write(lines)

CRYST1   17.969   15.562   14.672  90.00  90.00  90.00 P 1           1 
- box info added 99 times
